# Hito 1 Minería de Datos
## Datos astronómicos del SDSS DR 10

### Introducción
 - preguntas
 - motivación

### Exploración de datos
- estadísticas
- patrones iniciales

### Propuesta experimental
- estrategia metodologica

### Experimento preliminar
- implementación y análisis de resultados

### Anexos (opcional)
- graficos y códigos complementarios

In [ ]:
import gzip
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

# Definiendo las columnas pues los datos no las traen
columnas = ["objID", "RAdeg", "Dedeg", 
            "umag", "gmag", "rmag", "imag", "zmag", 
            "u'mag", "g'mag", "r'mag", "i'mag", "z'mag",
            "specID", "sublass", "z", "Qf"]

with gzip.open("dame_qso.dat.gz", "rt") as f:
    df = pd.read_csv(f, delim_whitespace=True, names=columnas, comment='#')
print(f"Cantidad de datos originalmente {df.shape}")
df.dropna(inplace=True)
print(f"Cantidad de datos sin nan {df.shape}")

Cantidad de datos originalmente (3602210, 17)
Cantidad de datos sin nan (163820, 17)
